# Интегрирование в системах компьютерной алгебры: 
## от реализации классических алгоритмов XIX века к новым геометрическим интеграторам 

Малых М.Д. (https://orcid.org/0000-0001-6541-6603)

Кафедра математического моделирования и искусственного интеллекта РУДН

URL: https://esystem.rudn.ru/faculty/ffmien/departments/kafedra-informacionnyh-tehnologii-5d56957fdac0c

Февраль 2025 г.

Вторая лекция посвящена исследованию символьных выражений, здесь рассмотрены различные символьные интеграторы, от самых первых Слегля и Мозеса до современных. 

## Лекция 2. Интегрирование элементарных функций

### Системы для символьных вычислений

В конце 1950-х годов в целях демонстарции возможностей компьютера по части мышления была предпринята попытка создания программы, которая вычисляла бы интегралы в символьном виде на уровне студента первого курса. Руководитель --- Марвин Минский.

Ссылки:
* Slagle J.R. A heuristic program that solve symbolic integration problem in freshman calculus, SAINT. MIT, 1961
* Moses J. Symbolic integration. MIT, 1967
* Moses J. Symbolic Integration: The Stormy Decade // Communications of the ACM. August 1971. Volume 14. Number 8

### Элементарные функции

Само определение элементарной функции в некотором смысле рекурсия. Элементарная функция -- это:
* число,
* буква (переменная),
* одна или несколько элементарных функций, связанных операцией зи списка элементарных операций.
Важна гипотеза о конечности числа операций, входящих в элементанную функцию.  

Элементарная функция --- черный ящик, про который мы знаем только две вещи:
* последнуюю операцию из списка элементарных операций
* операнды последней опеарции

Алгоритм дифференцирования и простейшие алгоритмы интегрирования --- рекурсии, использующие только две вспомогательные функции --- operator и operands. 

In [3]:
var('x')
(1+sin(x)).operator()

<function add_vararg at 0x7fbfdb3c18a0>

In [4]:
(1+sin(x)).operands()

[sin(x), 1]

Некоторые операции применяются к списку операндов, который имеет любую длину; некоторые -- к списку длины 2, некоторые к списку из 1 операнда.  

In [5]:
(2*sin(x)*cos(x)).operands()

[cos(x), sin(x), 2]

In [6]:
(cos(sin(x)+x^2)).operands()

[x^2 + sin(x)]

Вычитание в Sage нет, есть умножение на -1.

In [7]:
(2-sin(x)).operator()

<function add_vararg at 0x7fbfdb3c18a0>

In [8]:
(2-sin(x)).operands()

[-sin(x), 2]

Но есть возведение в степень

In [9]:
(x^x).operator()

<built-in function pow>

In [10]:
(x^x).operands()

[x, x]

Многие числа рассматриваются как составные выражения:

In [11]:
sqrt(2).operands()

[2, 1/2]

Список встроенных названий операций:

In [12]:
def names_of_operators():
    D={}
    D['+']=(1+x).operator()
    D['*']=(2*x).operator()
    D['/']=(2/x).operator()
    D['^']=(2^x).operator()
    D['ln']=(ln(x)).operator()
    D['exp']=(exp(x)).operator()
    return D


In [13]:
names_of_operators()['+']

<function add_vararg at 0x7fbfdb3c18a0>

### Дифференцирование

Правила дифференцирования. Пусть $u,v$ --- элементарные выражения, тогда 
1) $$
\quad d(u+v+\dots)=du+dv+\dots;
$$
2) $$
\quad d(uv\dots)=(v\dots)du+(u\dots)dv+\dots;
$$
3) $$
\quad d\frac{u}{v}=\frac{vdu-udv}{v^2};
$$
4) $$
\quad d(u^v)=de^{v \ln u}=vu^{v-1}du+u^v\ln(u) dv;
$$
5) $$
\quad d\ln u= \frac{du}{u};
$$
6) $$
\quad d\exp u= \exp u \cdot du.
$$

In [14]:
def mdiff(f,x):
    D=names_of_operators()
    if f==x:
        return 1
    elif f.operator()==D['+']:
        return sum([mdiff(g,x) for g in f.operands()])
    elif f.operator()==D['*']:
        P=f.operands()
        return sum([mdiff(p,x)*prod(P)/p for p in P])
    elif f.operator()==D['/']:
        [u,v]=f.operands()
        return (mdiff(u,x)*v- mdiff(v,x)*u)/v^2
    elif f.operator()==D['^']:
        [u,v]=f.operands()
        return v*u^(v-1)*mdiff(u,x)+ u^v*mdiff(v,x)*ln(u)
    elif f.operator()==D['ln']:
        [u]=f.operands()
        return mdiff(u,x)/u
    elif f.operator()==D['exp']:
        [u]=f.operands()
        return f*mdiff(u,x)
    else:
        return 0

In [15]:
var("x,y")
mdiff(2^x*x*ln(x),x).full_simplify()

(x*log(2) + 1)*2^x*log(x) + 2^x

In [16]:
diff(2^x*x*ln(x),x).full_simplify()

(x*log(2) + 1)*2^x*log(x) + 2^x

Теорема. Производная любого элементарного выражения является элементарным выражением, которое может быть найдено по указанному выше алгоритму за конечное число шагов. 

### Итегрирование

Слегль: интегрирование --- это стратегия, будут ветви и нужно вовремя отпиливать плохие ветки.

#### Опыты с таблицей интегралов

Чтобы проинтегрировать выражение по таблице интегралов, нужно сравнить его с эталонным выражением и, в случае успеха, написать ответ. Сравнение --- это аналог поиска по шаблону или регулярному выражению. В Sage он реализован в виде метода match. Допустим, что нам нужно опознать выражение вида $cx^a$. 

In [23]:
var('x,y')
a = SR.wild(0) 
c = SR.wild(1)
D = (2/x).match(c*x^a)

In [24]:
D

{$1: 2, $0: -1}

In [25]:
D[c]

2

In [26]:
(2*(x+y)/x).match(c*x^a)

{$1: 2*x + 2*y, $0: -1}

Начнем реализацию нашей таблицы интегралов с интегрирования степеней
$$
\int cx^a dx = 
\begin{cases}
\frac{с x^{a+1}}{a+1}, & a\not =-1\\
\ln |x|, & a=-1.
\end{cases}
$$
Вообще говоря, match справляется с задачей определения такого вида выражений. Однако match не подхватывает выражения, в которых константы равны 0 или 1. По этой причине особыми случаями для нас будут $a=0$, $a=1$ и $c=1$. 

In [27]:
def itable(f,x):
    if diff(f,x)==0:
        return f*x
    c=f/x
    if diff(c,x)==0:
        return c*x^2/2
    c = SR.wild(0)
    a = SR.wild(1) 
#1 and 2.) f=c*x^n
    g=x^a
    D=f.match(x^a)
    if D is not None and diff(D[a],x)==0:
        if D[a]==-1:
            return ln(abs(x))
        else:
            return 1/(D[a]+1)*x^(D[a]+1)
    D=f.match(c*g)
    if D is not None and diff(D[c],x)==0:
        return D[c]*itable(f/D[c],x)
#3.) f=c*a^x
    g=a^x
    D=f.match(g)
    if D is not None and diff(D[a],x)==0 and D[a]>0:
        return D[a]^x/ln(D[a])
    D=f.match(c*g)
    if D is not None and diff(D[c],x)==0:
        return D[c]*itable(f/D[c],x)
#4.) f=b*exp(x)
    b=f/exp(x)
    if diff(b,x)==0 :
        return b*exp(x)
#15. and 16.) f=c/(x^2+a)
    g=1/(x^2+a)
    D=f.match(g)
    if D is not None:
        if D[a]>0:
            return arctan(x/sqrt(D[a]))
        elif D[a]<0:
            return 1/2/sqrt(-D[a])*log((x - sqrt(-D[a]))/(x+sqrt(-D[a])))
    D=f.match(c*g)
    if D is not None and diff(D[c],x)==0:
        return D[c]*itable(f/D[c],x)
#16.1) f=c/(x^2-a)
    g=1/(x^2-a)
    D=f.match(g)
    if D is not None:
        if D[a]<0:
            return arctan(x/sqrt(-D[a]))
        elif D[a]>0:
            return 1/2/sqrt(D[a])*log((x - sqrt(D[a]))/(x+sqrt(D[a])))
    D=f.match(c*g)
    if D is not None and diff(D[c],x)==0:
        return D[c]*itable(f/D[c],x)
#16.2) f=c/(a-x^2)
    g=1/(a-x^2)
    D=f.match(g)
    if D is not None:
        return -itable(1/(x^2-D[a]),x)
    D=f.match(c*g)
    if D is not None and diff(D[c],x)==0:
        return D[c]*itable(f/D[c],x)

Примеры

In [28]:
itable(2/x,x)

2*log(abs(x))

In [29]:
itable(pi,x)

pi*x

In [30]:
itable(2*x^2,x)

2/3*x^3

In [31]:
itable(2/y,y)

2*log(abs(y))

In [32]:
itable(2*y/x^2,x)

-2*y/x

При реализации 3 и 4 строк таблицы 
$$
\int ca^x dx = 
\begin{cases}
\frac{ca^x}{\ln a} , & a\not =e\\
e^x, & a=e.
\end{cases}
$$
проявляется еще одна проблема в работе метода match: система воспринимает выражение $e^x$ как $\exp(x)$ и последняя операция здесь --- взятие экспоненты, а не возведение в степень ^.

In [34]:
itable(2^x,x)

2^x/log(2)

In [35]:
itable(e^x,x)

e^x

П. 15 и 16 таблицы гласят
$$
\int \frac{dx}{x^2+a^2} =\frac{1}{a} \arctan \frac{x}{a}
$$
$$
\int \frac{dx}{x^2-a^2} =\frac{1}{2a} \ln \left|\frac{x-a}{x+a} \right|
$$
Мы реализовали формулу
$$
\int \frac{c dx}{x^2+a} = 
\begin{cases}
\frac{c}{\sqrt{a}} \arctan \frac{x}{\sqrt{a}} , & a>0\\
\frac{c}{2\sqrt{-a}} \ln \left|\frac{x-\sqrt{-a}}{x+\sqrt{-a}} \right|, & a<0.
\end{cases}
$$
В нашу реализацию мы добавили вентиль по знаку $a$. Тут следует подчеркнуть, что система может сравнивать с нулем не только алгебраические числа, но и константы типа $\pi$ и $e$, а также переменные, относительно знака  которых были сделаны предположения. 

In [37]:
itable(1/(x^2+5),x)

arctan(1/5*sqrt(5)*x)

In [38]:
itable(-2/(x^2-3),x)

-1/3*sqrt(3)*log((x - sqrt(3))/(x + sqrt(3)))

In [39]:
itable(1/(x^2+pi),x)

arctan(x/sqrt(pi))

In [40]:
var('a')
assume(a>0)
itable(1/(x^2+a^2),x)

arctan(x/a)

В Sage нет настоящего вычетания, поэтому

In [41]:
(1/(x^2-pi)).match(1/(x^2+a))

Мы добавили отдельно ветку с вычетанием

In [43]:
itable(3/(2-x^2),x)

-3/4*sqrt(2)*log((x - sqrt(2))/(x + sqrt(2)))

In [44]:
itable(3/(x^2-pi^2),x)

3/2*log(-(pi - x)/(pi + x))/pi

#### Интегрирование подстановкой

Нам задано подынтегральное выражение в интеграле
$$
\int f(x)dx.
$$
* Посмотрев на $f$, мы выбираем подстановку $g$. Обычно $g$ --- это кусок выражения $f$. 
* Затем мы решаем уравнение
$$
u=g(x)
$$
относительно $x$ и выражаем через $u$
$$
\frac{f(x)}{g'(x)}=h(u).
$$
* Ищем интеграл в таблице или делаем новую подстановку.

Выражение $g$ входит в $f$, поэтому как правило дело сводится к тому, чтобы в выражении $h=f/g'$ кусок $g$ заменить на $u$. Sage поддерживает такую замену:

In [46]:
var('u')
sin(cos(x)).subs(cos(x)==u)

sin(u)

In [47]:
def integrate_by_subs(f,x,G=[]):
    F=itable(f,x)
    if F is not None:
        return F
    if G==[]:
        G=f.operands()
    k=0 # флаг, увеличивается, если есть невырожденные подстановки
    for g in G: 
        if diff(g,x)!=0 and g!=x:
            k=k+1
            var('_x')
            print('==subs.==')
            print(_x==g)
            h=(f/diff(g,x)).subs(g==_x)
            print(h)
            if diff(h,x)==0:
                H=itable(h,_x)
                if H is not None:
                    return H.subs(_x==g)
    if k==0:
        return None # все элементы списка G --- вырожденные подстановки
    L=[]
    for g in G:
        L=L+g.operands()
    return integrate_by_subs(f,x,G=L)

Пояснение. У функции два обязательных аргумента и один --- список подстановок $G$ "--- необязательный. Прежде всего функция пытается взять интеграл по таблице. Если это не получается и список подстановок $G$ пуст, то он заполняется операндами подынтегральной функции. Затем для каждого операнда $g$ из списка $G$ проверяется, не дает ли он вырожденную подстановку $x=$const или тривиальную подстановку $x=u$. Такие подстановки отбрасываются. Если других нет, то возвращается None. В противном случае вычисляется $h(u)$ и, если такой интеграл есть в таблице, функция возвращает первообразную. Если по таблице не удалось взять ни один из интегралов, то список операндов $f$ заменяется на список операндов операндов выражений, входящих в $G$, и повторяется применение функции с этим списком подстановок. 

In [48]:
integrate_by_subs(x*e^(x^2),x)

==subs.==
_x == e^(x^2)
1/2


1/2*e^(x^2)

In [49]:
integrate_by_subs(x*e^(pi*x^2),x)

==subs.==
_x == e^(pi*x^2)
1/2/pi


1/2*e^(pi*x^2)/pi

In [50]:
integrate_by_subs(x/(1+x^2),x)

==subs.==
_x == (1/(x^2 + 1))
-1/2*x^2 - 1/2
==subs.==
_x == x^2 + 1
1/2/_x


1/2*log(abs(x^2 + 1))

Проблемы:
* Никакой стратегии для вычисления интегралов не надо. Подстановка обычно одна единственная. 
* Если интеграл не берется, то это не значит, что его нельзя взять в элементарных функциях. Многие задачи требуют решения уранвения $u=g(x)$. 
* При реализации метода интегрирования по частям возникают бесконечные циклы. 

То, чему учат первокурсников, не является алгоритмами интегрирования. Это --- рецепты, которые можно реализовавывать по разному, но одинаково плохо. 

Ссылка:
* Малых М.Д., Севастьянов А.Л., Севастьянов Л.А. О СИМВОЛЬНОМ ИНТЕГРИРОВАНИИ В КУРСЕ МАТЕМАТИЧЕСКОГО АНАЛИЗА // Компьютерные инструменты в образовании. 2019. С. 94-106

### Метод Лиувилля

Теорема Лиувилля, 1833. Пусть $p,q$ -- многочлены. Интеграл 
$$
\int q(x)e^{p(x)} dx
$$
берется в элементарных функциях тогда и только тогда, когда существует многочлен $r$, такой, что
$$
\int q(x)e^{p(x)} dx = r(x)e^{p(x)} + C.
$$

Дифференцируя и сокращая на $e^p$, имеем
$$
q = r' + rp', 
$$
откуда
$$
\deg(r)=\deg(q)-\deg(p)+1
$$
Выяснить, существует ли такой многочлен, можно по методу неопределенных коэффциентов. 

In [51]:
def lintegral(f,x):
    qq = SR.wild(0)
    pp = SR.wild(1)
    D=(f.factor()).match(qq*e^pp)
    q=qq.subs(D)
    p=pp.subs(D)
    K=QQ[x]
    dr=K(q).degree()-K(p).degree()+1
    if dr<0:
        return False
    else:
        c=var(['c'+str(i) for i in range(dr+1)])
        r=sum([c[i]*x^i for  i in range(dr+1)])
        eqs=QQ[c][x](q-diff(r,x)-r*diff(p,x)).coefficients()
        teqs=triangulation([QQ[c](eq) for eq in eqs])
        if len(teqs)==dr+1:
            D=tsolve(teqs)
            return r.subs(D)*e^p
        else:
            return False 

In [55]:
load('gauss.sage')
lintegral(x^5*e^(x),x)

(x^5 - 5*x^4 + 20*x^3 - 60*x^2 + 120*x - 120)*e^x

In [56]:
lintegral(x*e^(x^3),x)

False

In [57]:
lintegral(x^2*e^(x^3),x)

1/3*e^(x^3)

Идея доказательства теоремы Лиувилля: первообразная --- аддитивная функция константы. 

Пусть 
$$
\int q(x)e^{p(x)} dx = F(x, e^p, e^r),
$$
тогда
$$
\int q(x)e^{p(x)} dx = F(x, e^p, Сe^r),
$$
но это невозможно. Пусть 
$$
\int q(x)e^{p(x)} dx = F(x, e^p, \ln r),
$$
тогда
$$
\int q(x)e^{p(x)} dx = F(x, e^p, \ln r + С),
$$
но тогда 
$$
\int q(x)e^{p(x)} dx = h(x, e^p) + A\ln r(x, e^p),
$$
Но тогда
$$
\int q(x)Ce^{p(x)} dx = h(x, Ce^p) + A\ln r(x, Ce^p).
$$
Это возможно лишь тогда, когда
$$
\int q(x)Ce^{p(x)} dx = h(x)e^p.
$$

Ссылки:
* Liouville J. Mémoire sur l'intégration d'une classe de fonctions transcendantes // Crelle's Journal, Bd. 13, pp. 93 - 118
* Manuel Bronstein. Symbolic Integration I. Transcendental Functions. Springer-Verlag Berlin Heidelberg, 2005
* Павлов Д.А. Символьное интегрирование. // Компьютреные инструменты в образовании. № 2. 2010. 

### Заключение

* Современные интеграторы основаны на методе Лиувилля, методе, который никогда не будут использовать люди, только компьютер.
* Ни один интегратор не дает гарантий, что невзятый интеграл не берется. 
* Имеется огромное слепое пятно --- интегрирование алгебарических функций. 

Проблема в теореме Абеля: $(x_1,y_1), \dots, (x_n, y_n)$ и $(x_1',y_1'), \dots, (x_n', y_n')$  --- нули и полюса радикала некоторой рациональной функции на кривой
$$
f(x,y)=0
$$
тогда и только тогда, когда
$$
\sum \limits_{i=1}^n \int \limits_{(x_i,y_i)}^{(x_i',y_i')} H_j(xy)dx = \frac{\omega_j}{k}, \quad j=1,\dots, p.
$$
При произвольном $k$ их оказывается слишком много!

Ссылка:
* Malykh, M.D., Sevastianov, L.A., Yu, Y. On symbolic integration of algebraic functions // Journal of Symbolic Computation Volume 104, 1 May 2021, Pages 563-579. https://doi.org/10.1016/j.jsc.2020.09.002




